<h1 style='text-align: center'>SQL Queries</h1>

## Getting Data From A SQL Database

### The Structure of a SQL Query

<img src='images/sql_statement.jpg'/>

#### SELECT

- What columns of data do we want? 
- Specify by column names or * (All) 

- Where are we getting our data from? 
- Specify by table name or use join statement to join multiple tables

##### JOIN 

- Use to join multiple tables 
- SELECT * FROM students join favorite_foods on students.id = favorite_foods.student_id
- SELECT * FROM students as s join favorite_foods as f on s.id = f.student_id

#### WHERE

- apply some filter to the data you are querying 
- define criteria for data that should selected from the database 
- SELECT * from students WHERE age > 10 

#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER you have made your initial query 
- When should we use HAVING instead of WHERE

In [1]:
import sqlite3

conn = sqlite3.connect('tutorial.db')
c = conn.cursor()

In [2]:
# Load students.json file

#your code here
import json
f=open('students.json','r')
data=json.load(f)

In [3]:
# iterate over the data and insert each student into the DB
data[0]

{'name': 'Sean Abu Wilson',
 'birthdate': '02/06',
 'siblings': 2,
 'Birthplace': 'Birmingham, AL',
 'yearsinnyc': 7.8,
 'favoritefood': 'guacamole'}

In [4]:
c.execute("drop table studentInfo")

In [5]:
conn.commit()

In [6]:
c.execute('''CREATE TABLE IF NOT EXISTS studentInfo (name TEXT, birthdate TEXT, siblings INTEGER, birthplace 
TEXT, yearsinnyc INTEGER, favoritefood TEXT)''')

conn.commit()

In [7]:
# dynamic data entry example
def dynamic_data_entry(person):
    c.execute("""INSERT INTO studentInfo (name, birthdate, siblings, birthplace, yearsinnyc, favoritefood) 
    VALUES (?, ?, ?, ?, ?, ?)""",
          (person['name'], person['birthdate'], person['siblings'], person['Birthplace'], person['yearsinnyc'], person['favoritefood']))
    
# c.close
# conn.close

In [8]:
for person in data: 
    dynamic_data_entry(person)

In [9]:
c.execute("""select * from studentInfo""").fetchall()

[('Sean Abu Wilson', '02/06', 2, 'Birmingham, AL', 7.8, 'guacamole'),
 ('Tawab', '04/08/1998', 4, 'Afghanistan', 0.5, 'pizza'),
 ('Bridget Boakye', '1990-07-13', 3, 'Accra', 15, 'Indian'),
 ('Cristina', '9/9/1990', 2, 'Spain', 6, 'Sushi'),
 ('Danny (Haoyang) Yu', '1997-03-19', 2, '1997-03-19', 0, 'Galbi'),
 ('Dustin_Breitner', '04/15/94', 2, 'New_York', 3, 'Pizza'),
 ('Gregory DeSantis', '1987-11-23', 2, 'Highland Park, NJ', 13, 'Pizza'),
 ('Harrison Miller', '10/2/85', 0, 'New Jersey', 0, 'Pizza'),
 ('Jeremy Owens', '03/30/1990', 0, 'Virginia', 6, 'Avocado'),
 ('Jeremy Reikes', '1-/12/1995', 1, 'New York City', 23, 'Pizza'),
 ('Jonathan Peck', '02-19-1994', 2, 'New York', 25, 'Moms food'),
 ('Minna F', '11-19-1996', 3, 'San Francisco', 1, 'sushi'),
 ('Nabil Abbas', '12-07-1994', 2, 'Long Island', 6, 'Gyro'),
 ('Natalie Del Rossi', '10-23-1994', 0, 'Brooklyn', 18, 'pizza'),
 ('Remy Canario', '10-11-1988', 0, 'New York', 28, 'toast'),
 ('Xristos Katsaros', '03-24-1986', 1, 'Chicago Heig

### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?
7. Which student was born closest to the cohort's graduation date?

In [10]:
c.execute("""select name from studentInfo""").fetchall()

[('Sean Abu Wilson',),
 ('Tawab',),
 ('Bridget Boakye',),
 ('Cristina',),
 ('Danny (Haoyang) Yu',),
 ('Dustin_Breitner',),
 ('Gregory DeSantis',),
 ('Harrison Miller',),
 ('Jeremy Owens',),
 ('Jeremy Reikes',),
 ('Jonathan Peck',),
 ('Minna F',),
 ('Nabil Abbas',),
 ('Natalie Del Rossi',),
 ('Remy Canario',),
 ('Xristos Katsaros',)]

In [11]:
c.execute("""select name, siblings from studentInfo 
where siblings=(select max(siblings) from studentInfo)""").fetchall()

[('Tawab', 4)]

In [12]:
c.execute("""select count(*) from studentInfo
where siblings = 0""").fetchall()

[(4,)]

In [13]:
c.execute("""select * from studentInfo
ORDER BY yearsinnyc ASC
limit 3""").fetchall()

[('Danny (Haoyang) Yu', '1997-03-19', 2, '1997-03-19', 0, 'Galbi'),
 ('Harrison Miller', '10/2/85', 0, 'New Jersey', 0, 'Pizza'),
 ('Tawab', '04/08/1998', 4, 'Afghanistan', 0.5, 'pizza')]

In [14]:
c.execute("""select * from studentInfo where birthplace = 'New York'""").fetchall()

[('Jonathan Peck', '02-19-1994', 2, 'New York', 25, 'Moms food'),
 ('Remy Canario', '10-11-1988', 0, 'New York', 28, 'toast')]

In [15]:
c.execute("""SELECT count(*) from studentInfo where birthplace in ('New York', 'New_York', 'Long Island', 'Brooklyn')""").fetchall()

[(5,)]

In [16]:
c.execute("""SELECT favoritefood , count(*) from studentInfo
group by favoritefood COLLATE NOCASE
having count(*) > 1""").fetchall()

[('Pizza', 7), ('sushi', 2)]

In [18]:
c.execute('''
SELECT name,
abs(julianday('2018-' || substr(birthdate,1,2) || '-' || substr(birthdate,4,2)) - julianday('2018-06-19'))
FROM studentInfo
ORDER BY 2
LIMIT 1
''').fetchall()

[('Bridget Boakye', None)]

In [19]:
c.execute('''
SELECT name,
substr(birthdate,1,2)
from studentInfo
''').fetchall()


[('Sean Abu Wilson', '02'),
 ('Tawab', '04'),
 ('Bridget Boakye', '19'),
 ('Cristina', '9/'),
 ('Danny (Haoyang) Yu', '19'),
 ('Dustin_Breitner', '04'),
 ('Gregory DeSantis', '19'),
 ('Harrison Miller', '10'),
 ('Jeremy Owens', '03'),
 ('Jeremy Reikes', '1-'),
 ('Jonathan Peck', '02'),
 ('Minna F', '11'),
 ('Nabil Abbas', '12'),
 ('Natalie Del Rossi', '10'),
 ('Remy Canario', '10'),
 ('Xristos Katsaros', '03')]

In [20]:
c.execute('''
SELECT name,
julianday(2018-06-19)
from studentInfo
''').fetchall()

[('Sean Abu Wilson', 1993.0),
 ('Tawab', 1993.0),
 ('Bridget Boakye', 1993.0),
 ('Cristina', 1993.0),
 ('Danny (Haoyang) Yu', 1993.0),
 ('Dustin_Breitner', 1993.0),
 ('Gregory DeSantis', 1993.0),
 ('Harrison Miller', 1993.0),
 ('Jeremy Owens', 1993.0),
 ('Jeremy Reikes', 1993.0),
 ('Jonathan Peck', 1993.0),
 ('Minna F', 1993.0),
 ('Nabil Abbas', 1993.0),
 ('Natalie Del Rossi', 1993.0),
 ('Remy Canario', 1993.0),
 ('Xristos Katsaros', 1993.0)]